In [ ]:
import pandas as pd
import os
import tqdm
import plotly.express as px
import numpy as np
os.chdir(r'D:\Projects\football-odds-analysis')
from utils.preprocess import preprocess_odds_results, preprocess_pipeline
from utils.arbitrage import get_arbitrage_bets
from utils.bet_payout_engine import get_bet_results
from input_variables import CO_CODES

PORTFOLIO_STARTING_CASH = 1000
START_DATE = '2018-01-01'
PROBABILITY_THRESHOLD = 0.98

odds_results_data = preprocess_odds_results('all_avail_games.csv')
bookmakers_data = preprocess_pipeline('all_avail_games.csv')

bookmakers_odds = odds_results_data['Odds']
game_results = odds_results_data['Results']

arbitrage_bets = get_arbitrage_bets(bookmakers_odds, PROBABILITY_THRESHOLD)

In [ ]:
all_games_dates = game_results['Date'][game_results['Date'] > START_DATE].sort_values().unique()
portfolio_value = {all_games_dates[0]: PORTFOLIO_STARTING_CASH}

for i, date in tqdm.tqdm(enumerate(all_games_dates)):
    if i == 0:
        continue
    todays_portfolio_value = portfolio_value[all_games_dates[i-1]]
    dates_games = game_results[game_results['Date'] == date]
    dates_games_signals = dates_games[dates_games.index.isin(list(arbitrage_bets.keys()))]
    
    if len(dates_games_signals) > 0:
        per_set_cash = todays_portfolio_value/len(dates_games_signals)
        game_payout = 0
        for game in dates_games_signals.index:
            bets = arbitrage_bets[game]
            for cb in bets[0]:
                company = cb[:-1]
                bet_side = cb[-1]
                bet_payoff = get_bet_results(bookmakers_data, game, bet_side, CO_CODES[company], per_set_cash * (bets[1][bet_side]))
                game_payout = game_payout + bet_payoff
        
        portfolio_value[date] = game_payout
        continue
    portfolio_value[date] = todays_portfolio_value

portfolio = pd.DataFrame.from_dict([portfolio_value]).T
portfolio = portfolio.sort_index()
portfolio.columns = ['Betting_Portfolio_Value']

In [ ]:
fig = px.line(x = portfolio.index ,
              y = portfolio['Betting_Portfolio_Value'],
              title = f'Cumulative Value of Betting Strategy Starting with {str(int(PORTFOLIO_STARTING_CASH))}$')
fig.show()

In [ ]:
fig = px.line(x = portfolio.index ,
              y = np.log(portfolio['Betting_Portfolio_Value']),
              title = f'Log of Cumulative Value of Betting Strategy Starting with {str(int(PORTFOLIO_STARTING_CASH))}$')
fig.show()